# Exercise 4: Groundedness and Relevance

1. Create a set of relevance and groundedness tests using the provided documents.
    - Review the documents and create a set of prompts which extract specific facts from those documents.
    Open [JSON prompt file](relevance_and_groundedness_prompts.json) and add prompts (question) in input field.
    - Create a set of expected results as ground truth for those prompts.
    Open [JSON prompt file](relevance_and_groundedness_prompts.json) and add expected results in expected_output field.

Run the tests against each target RAG system.
    - There are 'openai' and 'llama3' provider available.
    - in the below cell, change custom_test = TestSet(prompts, feedbacks, name="Exercise4", default_provider="openai") openai to llama3 if you wish to evaluate with llama3.

RAG execution test files - and observe the results carefully and evaluate which RAG system performs better.
1. [openai](grounded_relevance_openai.py)
2. [llama3](grounded_relevance_llama3.py)

Go to [RAG application config file](./../../llm_application/azure_information_assistant_accelerator/config.json) and change folder to change the documents being retrieved.
Note: "All" means all folders existing.

Now re-run the tests and observe how the results differ.
1. [openai](grounded_relevance_openai.py)
2. [llama3](grounded_relevance_llama3.py)

And examine the chain of thought responses for each metric.

In [ ]:
import sys
import os
# Add the parent directory to sys.path
sys.path.append(os.path.abspath(os.path.join(os.path.dirname('__file__'), '..', '..')))

from llm_application.azure_information_assistant_accelerator.wrapper import rag_chain
from kjr_llm.targets import CustomTarget
from kjr_llm.app import App
from kjr_llm.tests import TestSet
from kjr_llm.targets import Target

from typing import List

from kjr_llm.prompts import PromptSet
from trulens_eval import Select
from kjr_llm.metrics import (
    Groundedness, 
    AnswerRelevance,
    ContextRelevance,
    GroundTruthAgreement
)


# Set up the test application
app = App(app_name="RAG_Application", reset_database=True)

# Define the target of our tests
target: Target = CustomTarget(rag_chain)

# Load our custom inputs - change the name of this file if you have prepared another prompts.
prompts = PromptSet.from_json_file("test_prompts.json")

# Import and instantiate feedback metrics
query_path = Select.Record.app.query.args.query
context_path = Select.Record.app.retrieve.rets[:]

# Using custom TestSet
# comment and uncomment the feedback you wish to evaluate
feedbacks = [
    Groundedness(context_path),
    ContextRelevance(query_path, context_path),
    AnswerRelevance(),
    GroundTruthAgreement(prompts)
]

# Define our test set - switch comment to evaluate llama3.
custom_test = TestSet(prompts, feedbacks, name="Exercise4-openai", default_provider="openai")
# custom_test = TestSet(prompts, feedbacks, name="Exercise4-llama3", default_provider="llama3")

# Evaluate our test set
result = custom_test.evaluate(target)

# Run the test dashboard to evaluate results
app.run_dashboard()

In [ ]:
# run this cell to stop the dashboard
app.stop_dashboard()